In [5]:
import torch
import torch.nn as nn

In [9]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
output = cos(input1, input2)

In [13]:
a = cos(torch.tensor([0.0]), torch.tensor([0.0]))

In [17]:
torch.mean(torch.tensor([0.0]))

tensor(0.)

In [1]:
from transformers import AutoTokenizer

tokenizer_model_name_path="hfl/chinese-roberta-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name_path)


In [18]:
tokenizer.decode([103])

'[MASK]'

In [19]:
import pickle

with open("tmp.pkl", "rb") as f:
    score = pickle.load(f)


In [31]:
print(score.shape)

torch.Size([52, 21128])


In [37]:
topk3 = score.topk(3, dim=1)[-1]

In [38]:
topk3

tensor([[2769,  511,  872],
        [ 872, 2644, 2769],
        [1962,  812, 2769],
        [ 106,  131,  117],
        [2769, 6443, 2218],
        [3221, 1373, 4263],
        [2476, 2769, 2484],
        [4263,  136, 2695],
        [3152, 2769,  872],
        [ 511, 8013,  117],
        [2769,  511,  872],
        [ 511, 8013,  106],
        [ 511,  106,  136],
        [ 511,  106, 8013],
        [ 511,  106, 8013],
        [ 511, 8013,  106],
        [ 511,  106, 8013],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 106, 2769,  511],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 872, 2769, 6468],
        [2769,  872, 6468],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 136,  511,  106],
        [ 511,  106, 8013],
        [ 511, 8013,